# Quickstart

This notebook provides a quick introduction in using Carsus with the SQL interface.

## Creating a database

To start working with a database you need to initialize it. This requires an `url`, for example `sqlite:///path/to/example.db`. 

In this guide we'll use a in-memory `sqlite` database, so we leave the `url` empty:

In [ ]:
from carsus import init_db
session = init_db()

Now, we have an SQLAlchemy `Session` object named `session`. We can use `session` to make simple queries. As you can see from the output, some basic atomic data has been ingested into our database. 

Let's examine it:

In [ ]:
from carsus.model import Atom

print('Number of Atoms in the database: {}'.format(session.query(Atom).count()))
si = session.query(Atom).get(14) 
print('Symbol: {}, atomic_number: {}, name: {}'.format(si.symbol, si.atomic_number, si.name))

So, our database already contains basic information about atoms. But this information has not been commited to the database yet. It is always **your responsibility** to commit the data!

In [ ]:
session.commit()

## Atomic weights and ionization energies

To store more interesting data we are going to use **ingesters**. For each data source supported by Carsus there is a corresponding **ingester class**. 

For example, to ingest atomic weights from the NIST Atomic Weights and Isotopic Compositions database:

In [ ]:
from carsus.io.nist import NISTWeightsCompIngester

weightscomp_ingester = NISTWeightsCompIngester(session)
weightscomp_ingester.ingest()

session.commit()

In [ ]:
print(si.weights)

Then continue with ionization energies:

In [ ]:
from carsus.io.nist import NISTIonizationEnergiesIngester

ioniz_energies_ingester = NISTIonizationEnergiesIngester(session, spectra='H-Mg')
ioniz_energies_ingester.ingest(ionization_energies=True, ground_levels=True)

session.commit()

## Levels, lines and collisions

Currently, Carsus supports two sources of energy levels and transition lines: the Robert Kurucz’s line list (GFALL) and the Chianti Database.


### GFALL

In [ ]:
from carsus.io.kurucz import GFALLIngester

gfall_url = 'http://kurucz.harvard.edu/linelists/gfall/gfall.dat'
gfall_ingester = GFALLIngester(session, fname=gfall_url, ions='H-Mg')
gfall_ingester.ingest(levels=True, lines=True)

session.commit()

### CHIANTI

You can ingest `levels`, `lines` and `collisions` from the Chianti Database.

In [ ]:
from carsus.io.chianti_ import ChiantiIngester

chianti_ingester = ChiantiIngester(session, ions='O 0-2')
chianti_ingester.ingest(levels=True, lines=True, collisions=True)

session.commit()

## Zeta data

In [ ]:
import os
import carsus
from carsus.io.zeta import KnoxLongZetaIngester

zeta_url = 'https://raw.githubusercontent.com/tardis-sn/carsus/master/carsus/data/knox_long_recombination_zeta.dat'
zeta_ingester = KnoxLongZetaIngester(session, zeta_url)
zeta_ingester.ingest()

session.commit()

## Save to HDF5

Once you have a database it is very easy to create atomic data files for TARDIS.

To do this, you need to use a special class called `AtomData`. This class takes `session` as its first argument; other important parameters are `selected_atoms` and `chianti_ions` (if any). 

Only data for `selected_atoms` will be stored in the output DataFrames and levels and lines for `chianti_ions` will be taken from the data source with the same name.

In [ ]:
from carsus.io.output import AtomData
atom_data = AtomData(session, 
                     selected_atoms='H-Mg', 
                     chianti_short_name='chianti_v9.0.1', 
                     chianti_ions='O 0-2')

<div class="alert alert-warning">

**NOTE:** if `chianti_short_name` does NOT match with your current version of Chianti DB the above command will fail.

</div>


The stored `DataFrames` can be easily accessed as attributes of `atom_data`:

In [ ]:
atom_data.atom_masses_prepared

In [ ]:
atom_data.levels_prepared.head(20)

In [ ]:
atom_data.macro_atom_prepared.head(20)

Finally, to create an HDF5 file you need to use the `to_hdf()` method.

In [ ]:
atom_data.to_hdf('example_store.h5', 
                 store_atom_masses=True, 
                 store_ionization_energies=True, 
                 store_levels=True, 
                 store_lines=True,
                 store_collisions=True,
                 store_macro_atom=True,
                 store_zeta_data=True)

You are done! Now you can use the created HDF5 atomic file to run TARDIS simulations.

In [ ]:
# Hidden cell
!rm example_store.h5